# Make predictions from INPUT_FOLDER

Note that we assume your input data is stored as **.h5 files**, which contain both raw data (image) and its mask (label needed for metrics computations). The input data is in INPUT_FOLDER. It will be created by the first cell.

We return new h5 files, with "raw", "label" and "pred" as keys. The output data will be in a new folder OUTPUT_FOLDER.

Enable GPU acceleration in "Notebook parameters" for faster predictions.

In [ ]:
INPUT_FOLDER = '/content/to_predict/' # data to predict folder
OUTPUT_FOLDER = '/content/predictions/'
PRETRAINED_MODEL_PATH = '/content/drive/MyDrive/6_aneurysm_segmentation/pretrained_models/iou0.40-3D_model_resnet18_Noneweights_100epochs_jaccard_finetuned_augmentTrue.best.hdf5'
COMPUTE_METRICS = True

# This will create a folder in the root /content/ folder. Place your .h5 files here
# We also create the output one. Your segmented data will be here.
import os
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

## Imports and Drive mount

In [ ]:
!pip install segmentation-models-3D --quiet
!pip install patchify --quiet

In [ ]:
# connect your drive to the session
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/6_aneurysm_segmentation/

In [ ]:
# import librairies
import numpy as np
import tensorflow as tf
import h5py
from tqdm import tqdm
from utils import load_data_from, analytics

In [ ]:
# load pretrained model
print(f"Reload from : {PRETRAINED_MODEL_PATH}")
model = tf.keras.models.load_model(PRETRAINED_MODEL_PATH, compile=False)

The next cell runs the entire data loading, model loading, predictions and export process. Computing metrics is not mendatory but useful to evaluate the performance of the model.

In [ ]:
# load data from INPUT_FOLDER
raw_data, labels, names = load_data_from(INPUT_FOLDER)

# predict
X_to_predict = np.expand_dims(raw_data, axis=-1)
y_pred = model.predict(X_to_predict, batch_size=1) # 1 to avoid OOM, but more is ok

# convert to binary
THRESHOLD = 0.5
y_pred01 = (y_pred > THRESHOLD).squeeze()

print('Predictions complete.')

if COMPUTE_METRICS:
    # can take a few seconds, fill free to remove if you don't care about metrics
    analytics(np.array(labels, dtype='float32'), y_pred01)

# convert to uint8 to match the initial format of the masks
y_pred01 = y_pred01.astype(np.uint8)

# save into OUTPUT_FOLDER
for i, filename in enumerate(names):
    raw = raw_data[i]
    label = labels[i]
    y_pred = y_pred01[i]

    h5f = h5py.File(f'{OUTPUT_FOLDER}{filename}', 'w')

    h5f.create_dataset('raw', data=raw)
    h5f.create_dataset('label', data=label)
    h5f.create_dataset('pred', data=y_pred)

    h5f.close()

print('\nExport complete.')